In [1]:
import pandas as pd
import numpy as np
import sys
import pickle

In [2]:
from pitched_recommend import Recommender

In [13]:
playlists_data = pd.read_csv('/Users/meshchd/Downloads/recommender_browse_recommendations_playlists_data.csv')
playlists_list = pd.read_csv('/Users/meshchd/Downloads/recommender_browse_recommendations_playlists_list.csv')

In [14]:
playlists_list.head()

,territory,category_id,Position,playlist_uri,playlist_name
0,TR,pop,4,spotify:playlist:37i9dQZF1DWXoHqNlfcLJb,OLAY
1,TR,pop,6,spotify:playlist:37i9dQZF1DX9G2MXp8YYmm,Lokal
2,AT,pop,2,spotify:playlist:37i9dQZF1DX2YSAZIuAiB1,Swag!
3,CH,pop,14,spotify:playlist:37i9dQZF1DX2YSAZIuAiB1,Swag!
4,DE,pop,2,spotify:playlist:37i9dQZF1DX2YSAZIuAiB1,Swag!


In [15]:
playlists_data.head()

,playlist_name,listeners,country_owner,playlist_owner,ranking,pl_ranking,playlist_uri
0,Today's Top Hits,4110687,US,spotify,1,1,spotify:playlist:37i9dQZF1DXcBWIGoYBM5M
1,RapCaviar,2679420,US,spotify,1,1,spotify:playlist:37i9dQZF1DX0XUsuxWHRQd
2,Summer Hits,2149595,US,spotify,1,1,spotify:playlist:37i9dQZF1DX1gRalH1mWrP
3,Mega Hit Mix,1943838,US,spotify,1,1,spotify:playlist:37i9dQZF1DXbYM3nMM0oPk
4,Esquenta Sertanejo,1822133,BR,spotify,1,1,spotify:playlist:37i9dQZF1DXdSjVZQzv2tl


In [23]:
def get_browse_playlists(playlists_data, playlists_list):

    playlists_data['long_uri'] = ['spotify:user:'+str(x) for x in playlists_data.playlist_uri]

    data = playlists_list.merge(playlists_data, how='left', left_on='playlist_uri', right_on='long_uri')

### Get the 'same_as_country' column

    data['same_as_country'] = [1 if x==y else 0 for (x,y) in zip(data.territory,data.country_owner)]

### Get the 'count_appearance' column

    playlist_count = data.playlist_uri_x.value_counts()
    playlist_count = pd.DataFrame(playlist_count).reset_index().rename(columns={'index':'playlist_uri',
                                                                            'playlist_uri_x':'count_appearance'})
    data = data.merge(playlist_count, how='left', left_on = 'playlist_uri_x', right_on = 'playlist_uri')

### Get the 'local' column

    data['local'] = [1 if x<20 else 0 for x in data.count_appearance]

### Get the 'mean_listeners_cat_terr' column

    by_cat_terr = data.groupby(by=['territory','category_id']).listeners.mean()
    df_cat_terr = pd.DataFrame(by_cat_terr).reset_index()
    df_cat_terr = df_cat_terr.rename(columns = {'listeners':'mean_listeners_cat_terr'})

    data = data.merge(df_cat_terr, how='left', left_on = ['territory','category_id'], 
                      right_on = ['territory','category_id'])

### Get the 'popular' column

    data['popular'] = [1 if x>y else 0 for (x,y) in zip(data.listeners,data.mean_listeners_cat_terr)]

### Get the 'score' column with the overall score
    data['score'] = data.same_as_country + data.local + data.popular

    cols = ['territory', 'category_id', 'Position', 'playlist_uri',
       'playlist_name_x', 'listeners', 'country_owner', 'count_appearance',
        'mean_listeners_cat_terr', 'same_as_country', 'local', 'popular',
        'score']


    data = data[cols].sort_values(by=['territory','category_id','score'], ascending=False)
    data_final = pd.DataFrame(data.groupby(by=['territory','category_id']).head(10).reset_index())
    
    return data_final

In [24]:
playlists = get_browse_playlists(playlists_data,playlists_list)

In [25]:
playlists.head()

,index,territory,category_id,Position,playlist_uri,playlist_name_x,listeners,country_owner,count_appearance,mean_listeners_cat_terr,same_as_country,local,popular,score
0,63367,UY,workout,2,spotify:playlist:37i9dQZF1DX9ZKyQHcEFXZ,Para Entrenar,NaN,NaN,17,NaN,0,1,0,1
1,64260,UY,workout,1,spotify:playlist:37i9dQZF1DX4PvjnQT47nf,Ritmo de Selección,NaN,NaN,6,NaN,0,1,0,1
2,62323,UY,workout,11,spotify:playlist:37i9dQZF1DX4eRPd9frC1m,Hype,NaN,NaN,58,NaN,0,0,0,0
3,62407,UY,workout,13,spotify:playlist:37i9dQZF1DWSJHnPb1f0X3,Cardio,NaN,NaN,58,NaN,0,0,0,0
4,62445,UY,workout,7,spotify:playlist:37i9dQZF1DX70RN3TfWWJh,Workout,NaN,NaN,58,NaN,0,0,0,0


In [3]:
playlists=pd.read_csv('/Users/meshchd/Downloads/data_browse_recommend_browse_playlists.csv')
playlists.head()

,index,territory,category_id,Position,playlist_uri,playlist_name_x,listeners,country_owner,count_appearance,mean_listeners_cat_terr,same_as_country,local,popular,score
0,59840,UY,workout,2,spotify:playlist:37i9dQZF1DX9ZKyQHcEFXZ,Para Entrenar,NaN,NaN,17,NaN,0,1,0,1
1,60732,UY,workout,1,spotify:playlist:37i9dQZF1DX4PvjnQT47nf,Ritmo de Selección,NaN,NaN,6,NaN,0,1,0,1
2,58838,UY,workout,9,spotify:playlist:37i9dQZF1DX4eRPd9frC1m,Hype,NaN,NaN,58,NaN,0,0,0,0
3,58862,UY,workout,13,spotify:playlist:37i9dQZF1DWSJHnPb1f0X3,Cardio,NaN,NaN,58,NaN,0,0,0,0
4,58937,UY,workout,6,spotify:playlist:37i9dQZF1DX70RN3TfWWJh,Workout,NaN,NaN,58,NaN,0,0,0,0


In [4]:
playlists.shape

(19451, 14)

In [5]:
with open('/Users/meshchd/Downloads/saved_model.pkl', 'r+b') as pickle_file:
    model = pickle.load(pickle_file)

In [5]:
class MacOSFile(object):
    def __init__(self, f):
        self.f = f

    def __getattr__(self, item):
        return getattr(self.f, item)

    def read(self, n):
        if n >= (1 << 31):
            buffer = bytearray(n)
            pos = 0
            while pos < n:
                size = min(n - pos, 1 << 31 - 1)
                chunk = self.f.read(size)
                buffer[pos:pos + size] = chunk
                pos += size
            return buffer
        return self.f.read(n)

In [44]:
with open('/Users/meshchd/Downloads/saved_model.pkl', "rb") as fin:
    model = pickle.load(MacOSFile(fin))

In [6]:
with open('/Users/meshchd/Downloads/metadata_dict.pkl', 'rb') as pickle_file:
    metadata = pickle.load(pickle_file)

In [7]:
all_territories = list(playlists.territory.unique())

In [8]:
rec = []

In [9]:
df=playlists.copy()

In [10]:
for territory in all_territories:
    #print("Producing recommendations for ", territory)
    terr_cat = list(df[df.territory==territory].category_id.unique())
    for category in terr_cat:
        #print("Producing recommendations for category ", category)
        terr_cat_playlists = list(df[(df.territory==territory)&(df.category_id==category)].playlist_uri.unique())
        rec.extend(model.rec_multiple_playlists(terr_cat_playlists, model.playlists_sparse, territory=territory,
                                          category = category, N=150))

In [11]:
rec

[(1341, 2.00674, 'spotify:playlist:37i9dQZF1DWUI1rlvkdQnb', 'UY', 'workout'),
 (36562, 1.364663, 'spotify:playlist:37i9dQZF1DX35oM5SPECmN', 'UY', 'workout'),
 (67424,
  1.3539172,
  'spotify:playlist:37i9dQZF1DWTl4y3vgJOXW',
  'UY',
  'workout'),
 (660502,
  1.3167804,
  'spotify:playlist:37i9dQZF1DWTl4y3vgJOXW',
  'UY',
  'workout'),
 (240897,
  1.2642282,
  'spotify:playlist:37i9dQZF1DWTl4y3vgJOXW',
  'UY',
  'workout'),
 (2899, 1.2533103, 'spotify:playlist:37i9dQZF1DWTl4y3vgJOXW', 'UY', 'workout'),
 (151307,
  1.239949,
  'spotify:playlist:37i9dQZF1DWTl4y3vgJOXW',
  'UY',
  'workout'),
 (19414,
  1.2355981,
  'spotify:playlist:37i9dQZF1DX76Wlfdnj7AP',
  'UY',
  'workout'),
 (91682,
  1.2336547,
  'spotify:playlist:37i9dQZF1DX76Wlfdnj7AP',
  'UY',
  'workout'),
 (383186,
  1.2328124,
  'spotify:playlist:37i9dQZF1DWTl4y3vgJOXW',
  'UY',
  'workout'),
 (42556,
  1.2289557,
  'spotify:playlist:37i9dQZF1DWTl4y3vgJOXW',
  'UY',
  'workout'),
 (90601,
  1.2254059,
  'spotify:playlist:37i9d

In [30]:
for territory in all_territories:
    #print("Producing recommendations for ", territory)
    terr_cat = list(df[df.territory==territory].category_id.unique())
    for category in terr_cat:
        #print("Producing recommendations for category ", category)
        terr_cat_playlists = list(df[(df.territory==territory)&(df.category_id==category)].playlist_uri.unique())
        rec.extend(model.rec_multiple_playlists(terr_cat_playlists, model.playlists_sparse, territory=territory,
                                          category = category, N=150))

recommendations = [(model.isrcs[x[0]],float(x[1]),x[2],x[3],x[4]) for x in rec]
recs_final = [(x[0],x[1],metadata['track_artist'][x[0]], metadata['track_title'][x[0]], metadata['major_label'][x[0]], metadata['label_studio'][x[0]],
               metadata['original_release_date'][x[0]], metadata['genre_name'][x[0]], metadata['parent_genre_name'][x[0]], 
               x[2], x[3], x[4]) for x in list(recommendations)]

all_columns = ['isrc','score','artist','title','major_label','label_studio','original_release_date','genre_name',
               'parent_genre_name','seed_playlist_uri','territory','category']
rec_df = pd.DataFrame(recs_final, columns = all_columns)
rec_df_umg = rec_df[rec_df.major_label=='UMG']

final_columns = ['territory', 'category', 'score', 'isrc', 'seed_playlist_uri', 'artist', 'title', 'original_release_date']
rec_df_final = rec_df_umg[final_columns]

rec_df_final.to_csv('~/Downloads/manual_recommendations_20190408.csv', index=False, sep='\t')

In [12]:
recommendations = [(model.isrcs[x[0]],float(x[1]),x[2],x[3],x[4]) for x in rec]
recs_final = [(x[0],x[1],metadata['track_artist'][x[0]], metadata['track_title'][x[0]], metadata['major_label'][x[0]], metadata['label_studio'][x[0]],
               metadata['original_release_date'][x[0]], metadata['genre_name'][x[0]], metadata['parent_genre_name'][x[0]], 
               x[2], x[3], x[4]) for x in list(recommendations)]

all_columns = ['isrc','score','artist','title','major_label','label_studio','original_release_date','genre_name',
               'parent_genre_name','seed_playlist_uri','territory','category']
rec_df = pd.DataFrame(recs_final, columns = all_columns)
rec_df_umg = rec_df[rec_df.major_label=='UMG']

final_columns = ['territory', 'category', 'score', 'isrc', 'seed_playlist_uri', 'artist', 'title', 'original_release_date']
rec_df_final = rec_df_umg[final_columns]

rec_df_final.to_csv('~/Downloads/manual_recommendations_20190614.csv', index=False, sep='\t')

In [13]:
rec_df_final.head()

,territory,category,score,isrc,seed_playlist_uri,artist,title,original_release_date
0,UY,workout,2.006740,USUM71607007,spotify:playlist:37i9dQZF1DWUI1rlvkdQnb,Luis Fonsi,Despacito,2017-03-31
1,UY,workout,1.364663,USC4R1503363,spotify:playlist:37i9dQZF1DX35oM5SPECmN,Nathaniel Rateliff & The Night Sweats,S.O.B.,2015-12-16
8,UY,workout,1.233655,USUG11800073,spotify:playlist:37i9dQZF1DX76Wlfdnj7AP,DJ Snake,Magenta Riddim,2018-06-29
19,UY,workout,1.185287,USUG11500395,spotify:playlist:37i9dQZF1DX35oM5SPECmN,Of Monsters and Men,Crystals,2015-08-07
22,UY,workout,1.179605,USUM71019699,spotify:playlist:37i9dQZF1DWTl4y3vgJOXW,Fabolous,You Be Killin Em,2014-01-01


In [16]:
rec_df_final[rec_df_final.title=='Unknown']

,territory,category,score,isrc,seed_playlist_uri,artist,title,original_release_date


In [15]:
playlists[(playlists.territory=='DE')&(playlists.category_id=='rock')]

,index,territory,category_id,Position,playlist_uri,playlist_name_x,listeners,country_owner,count_appearance,mean_listeners_cat_terr,same_as_country,local,popular,score
14133,19392,DE,rock,1,spotify:playlist:37i9dQZF1DX4vth7idTQch,Rockhymnen,NaN,NaN,3,NaN,0,1,0,1
14134,19564,DE,rock,19,spotify:playlist:37i9dQZF1DX7fE85waV8CI,Klare Kante,NaN,NaN,3,NaN,0,1,0,1
14135,20550,DE,rock,14,spotify:playlist:37i9dQZF1DWZJhOVGWqUKF,Rock Rotation,NaN,NaN,3,NaN,0,1,0,1
14136,22052,DE,rock,34,spotify:playlist:37i9dQZF1DXbz5uvvU2C9n,Alternative Generation,NaN,NaN,4,NaN,0,1,0,1
14137,18705,DE,rock,29,spotify:playlist:37i9dQZF1DX1X7WV84927n,Hard Rock,NaN,NaN,113,NaN,0,0,0,0
14138,18769,DE,rock,2,spotify:playlist:37i9dQZF1DWWGFQLoP9qlv,Legendary,NaN,NaN,58,NaN,0,0,0,0
14139,18794,DE,rock,39,spotify:playlist:37i9dQZF1DWT2jS7NwYPVI,New Noise,NaN,NaN,58,NaN,0,0,0,0
14140,18905,DE,rock,21,spotify:playlist:37i9dQZF1DWWJOmJ7nRx0C,Rock Hard,NaN,NaN,71,NaN,0,0,0,0
14141,18960,DE,rock,12,spotify:playlist:37i9dQZF1DXcF6B6QPhFDv,Rock This,NaN,NaN,88,NaN,0,0,0,0
14142,19026,DE,rock,8,spotify:playlist:37i9dQZF1DX6xOPeSOGone,Soft Rock,NaN,NaN,58,NaN,0,0,0,0


In [14]:
rec_df_final.to_csv('/Users/meshchd/Downloads/browse_rec_20190715.csv', index=False, sep='\t')